In [10]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier,VotingClassifier,VotingRegressor,RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from xgboost import XGBRegressor,XGBRFRegressor
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC,SVR
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.preprocessing import StandardScaler
from seedtools import load_seed,dropper,mapper_auto

## CONTENTS 

-  VotingClassifier
-  Voting Regressor

In [11]:
cols = ["Age","EstimatedSalary"]
ss =  StandardScaler()

lr =  LogisticRegression()
rf  = RandomForestClassifier()
kn =  KNeighborsClassifier()
svc =  SVC()
svc_soft =  SVC(probability=True)
rfr = RandomForestRegressor()
svr =  SVR()
lr2 =  LinearRegression()
dtr =  DecisionTreeRegressor(max_depth=3)
xgbr =  XGBRegressor()
xgbrf =  XGBRFRegressor() 


estims =  [('lr',lr),('rf',rf),('kn',kn)] # for classification hard
estims2 =  [('lr',lr),('rf',rf),('svc',svc_soft)] # for classification soft 
estims3 =  [('LR',lr2),('rfr',rfr),('svr',svr)]  # for regression 
estims4 =  [('LR',lr2),('rfr',rfr),('dtr',dtr)]  # for regression 
estims5 =  [('dtr',dtr),('rfr',rfr),('xgbr',xgbr)]  # for regression 
estims6 =  [('xgbrf',xgbrf),('dtr',dtr),('xgb',xgbr)]

In [12]:
df = load_seed("Social_Network_Ads.csv","v3",True).data
for col  in cols:
    df[col]  = ss.fit_transform(df[col].values.reshape(-1,1))
    
x = df.drop(columns=["Purchased"]).values
y =  df["Purchased"].values

x_train,x_test,y_train,y_test =  train_test_split(x,y,test_size=0.2)
print("X train shape :",x_train.shape)
df.head(3)


X train shape : (320, 3)


,Gender,Age,EstimatedSalary,Purchased
0,0,-1.781797,-1.490046,0
1,0,-0.253587,-1.460681,0
2,1,-1.113206,-0.785290,0


## Voting Classifier


**Hard Voting**

In [13]:


voting_hard =  VotingClassifier(estimators=estims)
voting_hard.fit(x_train,y_train)
preds =  voting_hard.predict(x_test)
print("Score :",voting_hard.score(x_test,y_test))

Score : 0.9375


**Soft Voting**

In [14]:


voting_soft =  VotingClassifier(estimators=estims2,voting="soft")
voting_soft.fit(x_train,y_train)
preds =  voting_soft.predict(x_test)
print("Score :",voting_soft.score(x_test,y_test))

Score : 0.95


## Voting Regressor

In [15]:
cars  = load_seed("cars.csv","v2",quiet=True).data.dropna()[:9000]

cols_ =  ["odometer_value","engine_capacity"]
ss2 = StandardScaler() 

for col  in cols_:
    cars[col]  = ss2.fit_transform(cars[col].values.reshape(-1,1))


x =  cars.drop(columns=['price_usd']).values
y =  cars["price_usd"].values

x_train,x_test,y_train,y_test =  train_test_split(x,y,test_size=0.2)
print("X train shape :",x_train.shape)
cars.head(3)


X train shape : (7200, 10)


,transmission,odometer_value,engine_fuel,engine_has_gas,engine_type,engine_capacity,has_warranty,state,drivetrain,price_usd,is_exchangeable
0,0,-0.375676,0,0,0,0.965421,0,0,0,10900.0,0
1,0,0.381466,0,0,0,1.826107,0,0,0,5000.0,1
2,0,1.229466,0,0,0,0.965421,0,0,0,2800.0,1


In [16]:
voting_Reg =  VotingRegressor(estimators=estims5)
voting_Reg.fit(x_train,y_train)

preds =  voting_Reg.predict(x_test)

print("Score :",voting_Reg.score(x_test,y_test))
print("R2 Score :",r2_score(y_test,preds))
print("MSE :",mean_squared_error(y_test,preds))

Score : 0.6400043868288716
R2 Score : 0.6400043868288716
MSE : 10936909.02848146
